**Project Topic:** Weather Condition Prediction from Image

**_Abstract:_** Basically, problem is to recognise or predict weather condition(air condition) in given picture(image data) . In this notebook , I am predicting weather condition by using extracted useful features from image and advanced methods/algorithms.

In [1]:
!ln -s /content/drive/My\ Drive/ mydrive      

In [ ]:
import pandas as pd
import numpy as np
import os
import keras
import matplotlib.pyplot as plt
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam

In [144]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import cv2
from google.colab.patches import cv2_imshow
import math
import seaborn as sns
from keras import optimizers, Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten,Dropout,Activation
from keras.utils import to_categorical
from tqdm import tqdm
from sklearn.model_selection import train_test_split
%matplotlib inline

**Assigning Images and Labels for all classes**

In [145]:
image_paths=[]
labels=[]

In [146]:
cnt=0
flag=0
path_of_class0="/content/mydrive/new_weather_train_data/0"
for r, d, f in os.walk(path_of_class0):
    for file in f:
        image_paths.append(os.path.join(r, file))
        labels.append(0)

In [147]:
print(len(image_paths))
print(len(labels))

500
500


In [148]:
cnt=0
flag=0
path_of_class1="/content/mydrive/new_weather_train_data/1"
for r, d, f in os.walk(path_of_class1):
    for file in f:
        image_paths.append(os.path.join(r, file))
        labels.append(1)

In [149]:
print(len(image_paths))
print(len(labels))

1000
1000


In [150]:
cnt=0
flag=0
path_of_class2="/content/mydrive/new_weather_train_data/2"
for r, d, f in os.walk(path_of_class2):
    for file in f:
        image_paths.append(os.path.join(r, file))
        labels.append(2)

In [151]:
print(len(image_paths))
print(len(labels))

1500
1500


In [152]:
cnt=0
flag=0
path_of_class3="/content/mydrive/new_weather_train_data/3"
for r, d, f in os.walk(path_of_class3):
    for file in f:
        image_paths.append(os.path.join(r, file))
        labels.append(3)

In [153]:
print(len(image_paths))
print(len(labels))

2000
2000


In [154]:
cnt=0
flag=0
path_of_class4="/content/mydrive/new_weather_train_data/4"
for r, d, f in os.walk(path_of_class4):
    for file in f:
        image_paths.append(os.path.join(r, file))
        labels.append(4)

In [155]:
print(len(image_paths))
print(len(labels))

2500
2500


**Fuctions for feature Extraction**

In [157]:
def contrast(rgb_image):
    d_primes = []
    b_primes = []

    for i in np.arange(len(rgb_image)):
        for j in np.arange(len(rgb_image[i])):
            d_primes.append(min((rgb_image[i][j][0],rgb_image[i][j][1],rgb_image[i][j][2])))
            b_primes.append(max((rgb_image[i][j][0], rgb_image[i][j][1], rgb_image[i][j][2])))

    max_b = max(b_primes)
    avg_d = sum(d_primes) / len(d_primes)
    avg_b = sum(b_primes) / len(b_primes)
    contrast = avg_d - avg_b
    normalized_contrast = (contrast) / (255)

    return (normalized_contrast,contrast,max_b,avg_d,avg_b)

def brightness(rgb_image):
    brightness_list = []
    for i in np.arange(len(rgb_image)):
        for j in np.arange(len(rgb_image[i])):
            pixel_r = rgb_image[i][j][0]
            pixel_g = rgb_image[i][j][1]
            pixel_b = rgb_image[i][j][2]

            brightness = (0.299 * pixel_r) + (0.587 * pixel_g) + (0.114 * pixel_b)
            brightness_list.append(brightness)

    brightness = sum(brightness_list)/len(brightness_list)
    normalized_brightness = (brightness) / (255)

    return normalized_brightness


def haze(contrast,max_b,avg_d,avg_b,lamb=1/3):
    A = (lamb * max_b) + ((1 - lamb) * avg_b)
    x1 = (A - avg_d) / A
    x2 = contrast / A

    haze = math.exp((-0.5 * ((5.1 * x1) + (2.9 * x2))) + 0.2461) #normalized value

    return haze


def color_hist(rgb_image, numofbin = 256):
    histR, bins = np.histogram(rgb_image[:, 0], np.arange(0, numofbin + 1), density=True)
    histG, bins = np.histogram(rgb_image[:, 1], np.arange(0, numofbin + 1), density=True)
    histB, bins = np.histogram(rgb_image[:, 2], np.arange(0, numofbin + 1), density=True)
    hist = np.concatenate((histR, histG, histB), axis=0)
    return hist

def intensity_hist(rgb_image,white_threshold):
    rgb_image = cv2.cvtColor(rgb_image, cv2.COLOR_RGB2GRAY)
    white_pixels = 0
    black_pixels = 0
    for i in np.arange(rgb_image.shape[0]):
        for j in np.arange(rgb_image.shape[1]):
            if (rgb_image.item(i, j) < white_threshold):
                black_pixels += 1
            else:
                white_pixels += 1

    return (white_pixels / (black_pixels + white_pixels))

def sharpness(rgb_image):
    image = cv2.cvtColor(rgb_image,cv2.COLOR_RGB2GRAY)  # to grayscale
    array = np.asarray(image, dtype=np.int32)

    dx = np.diff(array)[1:, :]  # remove the first row
    dy = np.diff(array, axis=0)[:, 1:]  # remove the first column
    dnorm = np.sqrt(dx ** 2 + dy ** 2)

    sharpness = np.average(dnorm)
    normalized_sharpness = sharpness / 40

    return normalized_sharpness

**Taking each image and extracting features from it and appending it in training data.**

In [ ]:
x_data_gray=[]
contrast_array=[]
brightness_array=[]
haze_array=[]
color_hist_array=[]
intensity_hist_array=[]
sharpness_array=[]

cnt=0
for path in image_paths:
    img = cv2.imread(path,cv2.IMREAD_COLOR)

    (norm_cont,c,max_b,avg_d,avg_b) =contrast(img)
    contrast_array.append(norm_cont)

    brightness_value=brightness(img)
    brightness_array.append(brightness_value)

    haze_value=haze(c,max_b,avg_d,avg_b)
    haze_array.append(haze_value)

    color_hist_value=color_hist(img)
    color_hist_array.append(color_hist_value)

    intensity_hist_value=intensity_hist(img,175)
    intensity_hist_array.append(intensity_hist_value)

    sharpness_value=sharpness(img)
    sharpness_array.append(sharpness_value)

**Converting each feature in numpy array so that we can concatenate it later.**

In [159]:
contrast_nparray=np.array(contrast_array)
brightness_nparray=np.array(brightness_array)
haze_nparray=np.array(haze_array)
color_hist_nparray=np.array(color_hist_array)
intensity_hist_nparray=np.array(intensity_hist_array)
sharpness_nparray=np.array(sharpness_array)

In [160]:
print(contrast_nparray.shape)
print(brightness_nparray.shape)
print(haze_nparray.shape)
print(color_hist_nparray.shape)
print(intensity_hist_nparray.shape)
print(sharpness_nparray.shape)

(2500,)
(2500,)
(2500,)
(2500, 768)
(2500,)
(2500,)


In [161]:
contrast_nparray.reshape(-1,1)
print(contrast_nparray.shape)

(2500,)


**Concatenating all the extracted features**

In [162]:
x_data=np.concatenate((contrast_nparray[:,None],brightness_nparray[:,None],haze_nparray[:,None],color_hist_nparray,intensity_hist_nparray[:,None],sharpness_nparray[:,None]),axis=1)
print(x_data.shape)

(2500, 773)


In [163]:
y_data=np.array(labels)
y_data=y_data.reshape(-1,1)
print(y_data.shape)
y_data_reshaped=y_data.reshape(-1,1)
print(y_data_reshaped.shape)

(2500, 1)
(2500, 1)


**Splitting the data into training and testing data**

In [164]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_data,y_data_reshaped, test_size=0.2, random_state=42)

In [165]:
print("x_train shape : ",x_train.shape)
print("x_test shape  : ",x_test.shape)
print("y_train shape : ",y_train.shape)
print("y_test shape  : ",y_test.shape)

x_train shape :  (2000, 773)
x_test shape  :  (500, 773)
y_train shape :  (2000, 1)
y_test shape  :  (500, 1)


**Applying SVC**

In [166]:
# Applying SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(x_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Pipeline(memory=None,
         steps=[('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('svc',
                 SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None,
                     coef0=0.0, decision_function_shape='ovr', degree=3,
                     gamma='auto', kernel='rbf', max_iter=-1, probability=False,
                     random_state=None, shrinking=True, tol=0.001,
                     verbose=False))],
         verbose=False)

**Evaluation Metrics**<br><br>
**Precision** : The precision is the ratio tp / (tp + fp) where tp is the number of true positives and fp the number of false positives. The precision is intuitively the ability of the classifier not to label as positive a sample that is negative.
<br>
<br>
**Recall** : The recall is the ratio tp / (tp + fn) where tp is the number of true positives and fn the number of false negatives. The recall is intuitively the ability of the classifier to find all the positive samples.
<br> 
<br>
**F1 Score**: The F1 score can be interpreted as a weighted average of the precision and recall, where an F1 score reaches its best value at 1 and worst score at 0. The relative contribution of precision and recall to the F1 score are equal. <br>
The formula for the F1 score is:

F1 = 2 * (precision * recall) / (precision + recall)
<br>
<br>
**Accuracy** : Accuracy is the measurement used to determine which model is best at identifying relationships and patterns between variables in a dataset based on the input, or training, data.

In [167]:
y_pred= clf.predict(x_test)

In [168]:
from sklearn.metrics import accuracy_score
print("accuracy :",accuracy_score(y_test, y_pred))


accuracy : 0.452


In [169]:
from sklearn.metrics import f1_score
print("f1-score : ",end="")
print(f1_score(y_test, y_pred, average='macro'))

from sklearn.metrics import recall_score
print("recall : ",recall_score(y_test, y_pred, average='macro'))

from sklearn.metrics import precision_score
print("precision : ",end="")
precision_score(y_test, y_pred, average='macro')


f1-score : 0.45121536890076214
recall :  0.46064188822253344
precision : 

0.45458176185209975

**Random Forests :** Random forests or random decision forests are an ensemble learning method for classification, regression and other tasks that operate by constructing a multitude of decision trees at training time and outputting the class that is the mode of the classes (classification) or mean/average prediction (regression) of the individual trees.

**Applying Random Forests**

In [170]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(bootstrap=False,
                            max_leaf_nodes=None,
                            n_estimators=12,  # The number of trees in the forest
                            min_weight_fraction_leaf=0.0,
                            )

In [171]:
rf.fit(x_train, y_train)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


RandomForestClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=12,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [172]:
y_pred1=rf.predict(x_test)

**Evaluation Metrics**<br><br>
**Precision** : The precision is the ratio tp / (tp + fp) where tp is the number of true positives and fp the number of false positives. The precision is intuitively the ability of the classifier not to label as positive a sample that is negative.
<br>
<br>
**Recall** : The recall is the ratio tp / (tp + fn) where tp is the number of true positives and fn the number of false negatives. The recall is intuitively the ability of the classifier to find all the positive samples.
<br> 
<br>
**F1 Score**: The F1 score can be interpreted as a weighted average of the precision and recall, where an F1 score reaches its best value at 1 and worst score at 0. The relative contribution of precision and recall to the F1 score are equal. <br>
The formula for the F1 score is:

F1 = 2 * (precision * recall) / (precision + recall)
<br>
<br>
**Accuracy** : Accuracy is the measurement used to determine which model is best at identifying relationships and patterns between variables in a dataset based on the input, or training, data.

In [173]:
from sklearn.metrics import accuracy_score
print("accuracy :",accuracy_score(y_test, y_pred1))


accuracy : 0.426


In [174]:
from sklearn.metrics import f1_score
print("f1-score : ",end="")
print(f1_score(y_test, y_pred, average='macro'))

from sklearn.metrics import recall_score
print("recall : ",recall_score(y_test, y_pred, average='macro'))

from sklearn.metrics import precision_score
print("precision : ",end="")
precision_score(y_test, y_pred, average='macro')


f1-score : 0.45121536890076214
recall :  0.46064188822253344
precision : 

0.45458176185209975